In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Once executed, you'll see a link to authorize access to your Google Drive. Click the link, select your Google account, and grant the necessary permissions. After that, your Google Drive will be mounted at `/content/drive`.

In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta
import os
import matplotlib.pyplot as plt
import seaborn as sns

##**Step 1: To load the data**

In [ ]:

# -----------------------
# 1. Load data
# -----------------------

train_path = "/content/drive/MyDrive/Exl Eq 2021/EQ_2021_Data_Sample.csv"
pred_template_path = "/content/drive/MyDrive/Exl Eq 2021/EQ-2021-Submission-Template.csv"

df = pd.read_csv(train_path)
pred_df = pd.read_csv(pred_template_path)

# Ensure consistent column names
# Training target is 'confirmed_cases'; output column is 'Confirmed_cases'
df['date'] = pd.to_datetime(df['date'])
pred_df['date'] = pd.to_datetime(pred_df['date'])

# In Example 2, 'Confirmed_cases' is initially NaN – we'll fill with predictions
if 'Confirmed_cases' not in pred_df.columns:
    pred_df['Confirmed_cases'] = np.nan


/tmp/ipython-input-2140374289.py:8: DtypeWarning: Columns (31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(train_path)


In [ ]:
# =====================================================
# Basic Info & Missing Values Summary
# =====================================================
info_text = []

info_text.append("===== DATA SHAPE =====")
info_text.append(str(df.shape))

info_text.append("\n===== COLUMNS =====")
info_text.append(str(df.columns.tolist()))

info_text.append("\n===== DATA TYPES =====")
info_text.append(str(df.dtypes))

info_text.append("\n===== MISSING VALUES =====")
info_text.append(str(df.isna().sum().sort_values(ascending=False)))

In [ ]:
# Create output directory
os.makedirs("EDA_Output", exist_ok=True)

with open("EDA_Output/01_basic_info.txt", "w") as f:
    f.write("\n".join(info_text))

In [ ]:
# =====================================================
# Describe Numeric Stats
# =====================================================
df.describe().to_csv("EDA_Output/02_numeric_summary.csv")

# =====================================================
# Time Coverage
# =====================================================
date_min = df["date"].min()
date_max = df["date"].max()

with open("EDA_Output/03_date_range.txt", "w") as f:
    f.write(f"Date Range: {date_min} → {date_max}\n")
    f.write(f"Number of days: {(date_max - date_min).days}")

# =====================================================
# Distribution Plots for Key Columns
# =====================================================
key_cols = ["confirmed_cases", "deaths", "population"]

for col in key_cols:
    if col in df.columns:
        plt.figure(figsize=(8,5))
        sns.histplot(df[col].dropna(), kde=True)
        plt.title(f"Distribution of {col}")
        plt.savefig(f"EDA_Output/dist_{col}.png", dpi=150)
        plt.close()

# =====================================================
# Missing Value Heatmap (if dataset small enough)
# =====================================================
plt.figure(figsize=(10,6))
sns.heatmap(df.isna(), cbar=False)
plt.title("Missing Values Heatmap")
plt.savefig("EDA_Output/missing_heatmap.png", dpi=150)
plt.close()

# =====================================================
# Cases Over Time (Aggregate)
# =====================================================
if "confirmed_cases" in df.columns:

    daily = df.groupby("date")["confirmed_cases"].sum()

    plt.figure(figsize=(12,6))
    plt.plot(daily.index, daily.values)
    plt.title("Total Confirmed Cases Over Time (All Counties)")
    plt.xlabel("Date")
    plt.ylabel("Confirmed Cases")
    plt.grid(True)
    plt.savefig("EDA_Output/cases_over_time.png", dpi=150)
    plt.close()

# =====================================================
# Top 10 counties by cumulative cases
# =====================================================
if "confirmed_cases" in df.columns and "countyFIPS" in df.columns:

    top_counties = (
        df.groupby("countyFIPS")["confirmed_cases"]
        .max()
        .sort_values(ascending=False)
        .head(10)
    )

    top_counties.to_csv("EDA_Output/top10_counties.csv")

    plt.figure(figsize=(8,5))
    sns.barplot(x=top_counties.values, y=top_counties.index)
    plt.title("Top 10 Counties by Max Confirmed Cases")
    plt.xlabel("Confirmed Cases")
    plt.ylabel("County FIPS")
    plt.savefig("EDA_Output/top10_counties.png", dpi=150)
    plt.close()

# =====================================================
# Correlation Heatmap (numerical features)
# =====================================================
numeric_df = df.select_dtypes(include=[np.number])

plt.figure(figsize=(12,8))
sns.heatmap(numeric_df.corr(), annot=False, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.savefig("EDA_Output/correlation_heatmap.png", dpi=150)
plt.close()

# =====================================================
# Cases Growth Example For a Random County
# =====================================================
if "confirmed_cases" in df.columns:

    sample_county = df["countyFIPS"].dropna().astype(int).unique()[0]

    temp = df[df["countyFIPS"] == sample_county].sort_values("date")

    plt.figure(figsize=(12,6))
    plt.plot(temp["date"], temp["confirmed_cases"])
    plt.title(f"Cases Over Time for CountyFIPS {sample_county}")
    plt.xlabel("Date")
    plt.ylabel("Confirmed Cases")
    plt.grid(True)
    plt.savefig("EDA_Output/sample_county_trend.png", dpi=150)
    plt.close()

**KEY INSIGHTS FROM EDA**
1. Very large dataset

Dataset has 1,179,375 rows × 82 columns

 — plenty of signal for ML models.

2. Full pandemic coverage

Date range: 2020-01-22 → 2021-01-30 (374 days).

This gives us enough data for trend + lag modeling.

3. Confirmed cases show strong exponential growth

Total-case graph shows a classic exponential curve.
This means models MUST use:

Lag features

Log1p transform on target

Time features

4. HIGH missingness in many mobility + economic features

Example:

apple_mobility_transit missing in ~1.12M rows

YoY_Reopened_Seated_Diner_Data missing in 1M rows

Many Google mobility & electricity data columns missing heavily

These features cannot be used directly they will hurt the model.

5. Strong feature correlations in SD (hospitalization) features

Correlation heatmap shows:

Hospital metrics + COVID hospitalization indicators are highly cross-correlated

But they are NOT stable over time (not enough early values)

Because of this:

They may introduce leakage

Should be excluded unless lagged AND only from past dates

6. County examples show monotonic growth

The countyFIPS=1001 curve is a typical smooth cumulative increasing series — perfect for:

Gradient boosting

Or LSTM/seq2seq (if desired)

In [ ]:
!pip install lightgbm xgboost -q

##**Step 2 — Select Features to Keep & Columns to DROP Entirely**

We must avoid features with:
*   Extreme missing values
*   Post-fact hospital metrics (leakage danger)
*   Electricity data (not useful for prediction)

We WILL USE:

Core features

countyFIPS

stateFIPS

date

Engineered time features:

dayofweek

weekofyear

month

days_since_start

Lag features (very important):

confirmed_cases_lag1

confirmed_cases_lag7

confirmed_cases_lag14

new_cases_lag1

new_cases_lag7

These capture trend, velocity, and acceleration.

**Drop because of massive missingness (as seen in heatmap & missing report)**

Google mobility columns

Apple mobility columns

Electricity data

Air passenger counts

All _SD (hospitalization) real-time state-level metrics

Workforce/employment correlations

These are not robust enough, not consistently available, and some are post-fact leak features.

In [58]:
import pandas as pd
import numpy as np
from datetime import timedelta

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# For reproducibility
RANDOM_STATE = 42

In [59]:
# 3. Feature selection (drop bad / leaky columns)
# ============================================
# Target
TARGET_COL = "confirmed_cases"

# Columns we definitely KEEP (time-varying & important)
base_keep = [
    "countyFIPS",
    "stateFIPS",
    "date",
    "confirmed_cases",
    "deaths",
    "test_count",
    "test_rate",
    "new_test_rate",
    "new_test_count",
]

# We also keep all demographic "C_*" columns (static per county)
demo_cols = [c for c in df.columns if c.startswith("C_")]

# Columns we drop completely (high missing or potential leakage)
drop_prefixes = [
    "google_mobility_",
    "apple_mobility_",
    "Electricity_Sales_",
    "DOMESTIC_",
    "INTERNATIONAL_",
    "YoY_Reopened_",
    "initclaims_",
    "spend_all_cd",
    "merchants_all_cd",
    "revenue_all_cd",
]
drop_suffix = "_SD"   # most hospital real-time stats

cols_to_use = set(base_keep + demo_cols)

for col in df.columns:
    # skip target & date & ids (already in base_keep)
    if col in cols_to_use:
        continue

    # drop by prefix
    if any(col.startswith(p) for p in drop_prefixes):
        continue

    # drop by suffix
    if col.endswith(drop_suffix):
        continue

    # otherwise, keep (if numeric) – optional
    if pd.api.types.is_numeric_dtype(df[col]):
        cols_to_use.add(col)

cols_to_use = list(cols_to_use)

print("\nNumber of selected columns:", len(cols_to_use))
print("Example selected columns:", list(cols_to_use)[:20])

df = df[cols_to_use].copy()


Number of selected columns: 26
Example selected columns: ['C_WA_MALE', 'countyFIPS', 'new_test_count', 'C_PCTPOV517_2019', 'C_TOT_POP', 'test_count', 'C_MEDHHINC_2019', 'S_D_dly_new_test', 'C_PCTPOV017_2019', 'S_D_cummulative_test', 'C_M_Employed_corr', 'C_PCTPOVALL_2019', 'C_MinorityPCT', 'C_M_Labour_Force_corr', 'C_HispanicPCT', 'C_BlackPCT', 'confirmed_cases', 'deaths', 'C_M_Unemployment_Rate_corr', 'test_rate']


##**Step 3 — Build Training Dataset with Feature Engineering**

We create:

Lags per county

Time features

log1p target

Train/validation split based on time (NOT random)

In [60]:
# ============================================
# 4. Time features
# ============================================
def add_time_features(frame: pd.DataFrame) -> pd.DataFrame:
    frame = frame.copy()
    frame["dayofweek"] = frame["date"].dt.dayofweek
    frame["weekofyear"] = frame["date"].dt.isocalendar().week.astype(int)
    frame["month"] = frame["date"].dt.month

    start_date = frame["date"].min()
    frame["days_since_start"] = (frame["date"] - start_date).dt.days

    return frame

df = add_time_features(df)

# ============================================
# 5. Lag features (per county)
# ============================================
def add_lag_features(frame: pd.DataFrame,
                     group_col: str,
                     target_cols,
                     lags=(1, 7, 14)):
    frame = frame.sort_values([group_col, "date"]).copy()

    for t_col in target_cols:
        for lag in lags:
            frame[f"{t_col}_lag{lag}"] = (
                frame.groupby(group_col)[t_col].shift(lag)
            )

        # simple growth features for confirmed_cases
        if t_col == "confirmed_cases":
            frame["new_cases_lag1"] = (
                frame["confirmed_cases_lag1"] - frame["confirmed_cases_lag7"]
            )
            frame["new_cases_lag7"] = (
                frame["confirmed_cases_lag7"] - frame["confirmed_cases_lag14"]
            )

    return frame

lag_target_cols = ["confirmed_cases", "deaths", "test_count", "new_test_count"]
lag_target_cols = [c for c in lag_target_cols if c in df.columns]

df = add_lag_features(df, group_col="countyFIPS",
                      target_cols=lag_target_cols,
                      lags=(1, 7, 14))

# Drop rows where lag features are NaN (start of each county series)
lag_cols = [c for c in df.columns if "_lag" in c or "new_cases_lag" in c]
df = df.dropna(subset=lag_cols).reset_index(drop=True)

print("\nAfter adding lags, shape:", df.shape)



After adding lags, shape: (1135345, 44)


In [61]:
# ============================================
# 6. Train / validation split (time-based)
# ============================================
df = df.sort_values("date")

max_date = df["date"].max()
cutoff_date = max_date - pd.Timedelta(days=30)  # last 30 days as validation

train_df = df[df["date"] <= cutoff_date].copy()
val_df   = df[df["date"] >  cutoff_date].copy()

print("\nTrain date range:", train_df["date"].min(), "→", train_df["date"].max())
print("Val date range:", val_df["date"].min(), "→", val_df["date"].max())
print("Train size:", train_df.shape[0], "Val size:", val_df.shape[0])

# ============================================
# 7. Prepare X, y
# ============================================
# Features = all numeric columns except target & date
feature_cols = [
    c for c in df.columns
    if c not in [TARGET_COL, "date"]
       and pd.api.types.is_numeric_dtype(df[c])
]

print("\nNumber of feature columns:", len(feature_cols))

X_train = train_df[feature_cols].values
X_val   = val_df[feature_cols].values

# log1p transform of target for stability
y_train_log = np.log1p(train_df[TARGET_COL].values)
y_val_log   = np.log1p(val_df[TARGET_COL].values)


Train date range: 2020-02-05 00:00:00 → 2020-12-31 00:00:00
Val date range: 2021-01-01 00:00:00 → 2021-01-30 00:00:00
Train size: 1040995 Val size: 94350

Number of feature columns: 32


##**Step 4 — Choose Best Model**

Given dataset size and feature characteristics:

**Best classical model: LightGBM Regressor**

Handles millions of rows efficiently

Handles categorical IDs

Handles nonlinearities

Supports monotonic constraints if needed

**Alternative: XGBoost**

In [62]:
# ============================================
# 8. Define models A (LightGBM) & B (XGBoost)
# ============================================
model_a = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

model_b = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=RANDOM_STATE,
    tree_method="hist",   # fast on CPU
    n_jobs=-1,
)

# ============================================
# 9. Train both models
# ============================================
print("\nTraining Model A (LightGBM)...")
model_a.fit(X_train, y_train_log)

print("Training Model B (XGBoost)...")
model_b.fit(X_train, y_train_log)

# ============================================
# 10. Evaluate models on validation set
#      (back-transform from log to original scale)
# ============================================
def evaluate_model(name, model, X_val, y_val_log):
    y_pred_log = model.predict(X_val)
    y_pred = np.expm1(y_pred_log)
    y_true = np.expm1(y_val_log)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)

    # RMSLE using log1p again
    rmsle = np.sqrt(mean_squared_error(
        np.log1p(y_true),
        np.log1p(np.clip(y_pred, 0, None))
    ))

    print(f"\n===== {name} Evaluation on Validation Set =====")
    print(f"RMSE  : {rmse:,.2f}")
    print(f"MAE   : {mae:,.2f}")
    print(f"R^2   : {r2:,.4f}")
    print(f"RMSLE : {rmsle:,.4f}")

    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "rmsle": rmsle,
    }

metrics_a = evaluate_model("Model A (LightGBM)", model_a, X_val, y_val_log)
metrics_b = evaluate_model("Model B (XGBoost)",  model_b, X_val, y_val_log)

print("\n===== SUMMARY COMPARISON =====")
print("Model A (LightGBM)  → RMSE: {:.2f}, RMSLE: {:.4f}".format(
    metrics_a["rmse"], metrics_a["rmsle"]))
print("Model B (XGBoost)   → RMSE: {:.2f}, RMSLE: {:.4f}".format(
    metrics_b["rmse"], metrics_b["rmsle"]))


Training Model A (LightGBM)...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.931072 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7001
[LightGBM] [Info] Number of data points in the train set: 1040995, number of used features: 32
[LightGBM] [Info] Start training from score 4.341778
Training Model B (XGBoost)...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(



===== Model A (LightGBM) Evaluation on Validation Set =====
RMSE  : 13,459.42
MAE   : 1,193.06
R^2   : 0.7498
RMSLE : 0.1146

===== Model B (XGBoost) Evaluation on Validation Set =====
RMSE  : 11,717.63
MAE   : 991.70
R^2   : 0.8104
RMSLE : 0.0638

===== SUMMARY COMPARISON =====
Model A (LightGBM)  → RMSE: 13459.42, RMSLE: 0.1146
Model B (XGBoost)   → RMSE: 11717.63, RMSLE: 0.0638


In [63]:
model_b.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.8,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'feature_weights': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.05,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 8,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 300,
 'n_jobs': -1,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.8,
 'tree_method': 'hist',
 'validate_parameters': None,
 'verbosity': None}

##**Final Step to run this model on test**

In [ ]:
import pandas as pd
import numpy as np


In [ ]:

import pandas as pd
import numpy as np
from datetime import timedelta
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Try to reuse an in-memory model if present
MODEL_VAR_NAME = "model_b"  # we used this name earlier
USE_EXISTING_MODEL = MODEL_VAR_NAME in globals()

# File paths
TRAIN_PATH = "/content/drive/MyDrive/Exl Eq 2021/EQ_2021_Data_Sample.csv"        # historical data
PRED_PATH  = "/content/drive/MyDrive/Exl Eq 2021/EQ-2021-Submission-Template.csv"      # prediction template
OUTPUT_PATH = "EQ_2021_submission.xlsx"

TARGET_COL = "confirmed_cases"
OUTPUT_TARGET_COL = "Confirmed_cases"   # column name in Example 2

# XGBoost params (used if we train here)
XGB_PARAMS = {
    "n_estimators": 300,
    "learning_rate": 0.05,
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "objective": "reg:squarederror",
    "random_state": 42,
    "tree_method": "hist",
    "n_jobs": -1,
}

print("Loading data...")
hist_df = pd.read_csv(TRAIN_PATH)
pred_template = pd.read_csv(PRED_PATH)

hist_df["date"] = pd.to_datetime(hist_df["date"], errors="coerce")
pred_template["date"] = pd.to_datetime(pred_template["date"], errors="coerce")
hist_df = hist_df.sort_values(["countyFIPS", "date"]).reset_index(drop=True)

print(f"History rows: {len(hist_df):,}, Prediction rows: {len(pred_template):,}")

# -----------------------
# Feature selection logic (same as training pipeline)
# -----------------------
base_keep = [
    "countyFIPS",
    "stateFIPS",
    "date",
    "confirmed_cases",
    "deaths",
    "test_count",
    "test_rate",
    "new_test_rate",
    "new_test_count",
]

demo_cols = [c for c in hist_df.columns if c.startswith("C_")]

drop_prefixes = [
    "google_mobility_",
    "apple_mobility_",
    "Electricity_Sales_",
    "DOMESTIC_",
    "INTERNATIONAL_",
    "YoY_Reopened_",
    "initclaims_",
    "spend_all_cd",
    "merchants_all_cd",
    "revenue_all_cd",
]
drop_suffix = "_SD"

cols_to_use = set(base_keep + demo_cols)
for col in hist_df.columns:
    if col in cols_to_use:
        continue
    if any(col.startswith(p) for p in drop_prefixes):
        continue
    if col.endswith(drop_suffix):
        continue
    if pd.api.types.is_numeric_dtype(hist_df[col]):
        cols_to_use.add(col)

cols_to_use = list(cols_to_use)
hist_df = hist_df[cols_to_use].copy()

# Ensure pred_template has required id/date columns
for c in ["countyFIPS", "stateFIPS", "date"]:
    if c not in pred_template.columns:
        raise ValueError(f"Prediction template missing required column: {c}")

# Create pred_stub matching history columns (confirmed_cases set to NaN)
pred_stub = pred_template.copy()
pred_stub[TARGET_COL] = np.nan
for col in hist_df.columns:
    if col not in pred_stub.columns and col not in ["confirmed_cases"]:
        if col in ["countyFIPS", "stateFIPS", "date"]:
            continue
        pred_stub[col] = np.nan
pred_stub = pred_stub[hist_df.columns]

# -----------------------
# time features & lag helper
# -----------------------
def add_time_features(df):
    df = df.copy()
    df["dayofweek"] = df["date"].dt.dayofweek
    df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int)
    df["month"] = df["date"].dt.month
    start_date = df["date"].min()
    df["days_since_start"] = (df["date"] - start_date).dt.days
    return df

def compute_lags(df, group_col="countyFIPS", target_cols=None, lags=(1,7,14)):
    df = df.sort_values([group_col, "date"]).copy()
    tcols = target_cols or ["confirmed_cases"]
    for tcol in tcols:
        for lag in lags:
            df[f"{tcol}_lag{lag}"] = df.groupby(group_col)[tcol].shift(lag)
        if tcol == "confirmed_cases":
            df["new_cases_lag1"] = df["confirmed_cases_lag1"] - df["confirmed_cases_lag7"]
            df["new_cases_lag7"] = df["confirmed_cases_lag7"] - df["confirmed_cases_lag14"]
    return df

# -----------------------
# Build combined history + prediction stub to compute initial lag columns
# -----------------------
combined = pd.concat([hist_df, pred_stub], ignore_index=True)
combined = add_time_features(combined)
lag_target_cols = [c for c in ["confirmed_cases", "deaths", "test_count", "new_test_count"] if c in combined.columns]
combined = compute_lags(combined, group_col="countyFIPS", target_cols=lag_target_cols, lags=(1,7,14))

lag_cols = [c for c in combined.columns if "_lag" in c or "new_cases_lag" in c]
print(f"Computed initial lag columns: {lag_cols[:6]} ... total {len(lag_cols)}")

# Forward-fill historical lags so that first future day inherits last real history
combined[lag_cols] = combined.groupby("countyFIPS")[lag_cols].ffill()
# Fallback: counties with no history -> fill zeros
combined[lag_cols] = combined[lag_cols].fillna(0)

# -----------------------
# Determine feature columns (numeric excluding target & date)
# -----------------------
feature_cols = [c for c in combined.columns if c not in [TARGET_COL, "date"] and pd.api.types.is_numeric_dtype(combined[c])]
print(f"Feature cols count: {len(feature_cols)}")

# -----------------------
# Prepare training set (full history rows only)
# -----------------------
train_mask = combined.index < len(hist_df)
train_df = combined.loc[train_mask].copy()
# Drop rows that still have NaNs in lag cols (should be rare after ffill/fillna)
train_df = train_df.dropna(subset=lag_cols).reset_index(drop=True)

X_train = train_df[feature_cols].values
y_train_log = np.log1p(train_df[TARGET_COL].values)

# -----------------------
# Load or train model
# -----------------------
if USE_EXISTING_MODEL:
    print("Using existing in-memory model 'model_b'.")
    model = globals()[MODEL_VAR_NAME]
else:
    print("No existing model found — training final XGBoost on full history now...")
    from xgboost import XGBRegressor
    model = XGBRegressor(**XGB_PARAMS)
    model.fit(X_train, y_train_log)
    # store into globals so subsequent cells can reuse
    globals()[MODEL_VAR_NAME] = model
    print("Training complete.")

# -----------------------
# Recursive forecasting loop
# -----------------------
print("Starting recursive forecasting...")

# We'll iterate date-by-date over prediction stub dates (in ascending order)
future_dates = pred_template["date"].sort_values().unique()
if len(future_dates) == 0:
    raise ValueError("No future dates found in Example 2.")

# Prepare a working dataframe that we will append predictions to (start with combined up to last history row)
work_df = combined.loc[combined.index < len(hist_df)].copy()
work_df = work_df.sort_values(["countyFIPS", "date"]).reset_index(drop=True)

# We'll store predictions in list in same order as pred_template
predictions = []

Loading data...


/tmp/ipython-input-1096674268.py:33: DtypeWarning: Columns (31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  hist_df = pd.read_csv(TRAIN_PATH)


History rows: 1,179,375, Prediction rows: 47,130
Computed initial lag columns: ['confirmed_cases_lag1', 'confirmed_cases_lag7', 'confirmed_cases_lag14', 'new_cases_lag1', 'new_cases_lag7', 'deaths_lag1'] ... total 14
Feature cols count: 32
Using existing in-memory model 'model_b'.
Starting recursive forecasting...


In [ ]:
# We'll iterate through each date in the pred_template (assumes pred_template contains contiguous block of future days)
for cur_date in sorted(future_dates):
    # Build rows for all counties present in pred_template for cur_date
    # We need to get the pred_template rows for cur_date (preserve ordering)
    cur_rows = pred_template[pred_template["date"] == cur_date].copy()
    cur_rows = cur_rows.reset_index(drop=True)

    # Ensure all required numeric columns exist in cur_rows (from hist_df's columns)
    for col in hist_df.columns:
        if col not in cur_rows.columns and col not in ["confirmed_cases"]:
            # fill with NaN (these are usually demographics/other static columns)
            cur_rows[col] = np.nan

    # Align column order
    # Ensure cur_rows has all columns that hist_df has (fill missing ones with NaN)
    for col in hist_df.columns:
        if col not in cur_rows.columns:
            # initialize missing columns as NaN; confirmed_cases will be set later from predictions
            cur_rows[col] = np.nan

    # Now reorder to exactly match hist_df column order
    cur_rows = cur_rows[hist_df.columns]

    # Append to work_df temporarily so shifts compute properly if needed (but we'll compute lags directly)
    # Instead of recalculating all lags, we will compute required lag features for cur_rows from work_df history.
    # For each county in cur_rows, fetch last known values needed for lag1, lag7, lag14.
    needed = []
    for idx, row in cur_rows.iterrows():
        c = row["countyFIPS"]
        # slice work_df for this county
        county_hist = work_df[work_df["countyFIPS"] == c].sort_values("date")
        # prepare a dict to hold features
        feat_row = {}
        feat_row["countyFIPS"] = c
        feat_row["stateFIPS"] = row.get("stateFIPS", np.nan)
        feat_row["date"] = cur_date
        # time features
        feat_row["dayofweek"] = cur_date.dayofweek
        feat_row["weekofyear"] = int(cur_date.isocalendar()[1])
        feat_row["month"] = cur_date.month
        feat_row["days_since_start"] = (cur_date - combined["date"].min()).days

        # For each lag col we need to fill from county_hist if available, else use 0
        for tcol in lag_target_cols:
            for lag in (1,7,14):
                lag_name = f"{tcol}_lag{lag}"
                # we need value of tcol at date = cur_date - lag
                target_date = cur_date - pd.Timedelta(days=lag)
                # look for entry in work_df for the county with that date
                val = np.nan
                # prefer the exact date in work_df (which includes historical and prior predictions appended)
                mask = (work_df["countyFIPS"] == c) & (work_df["date"] == target_date)
                if mask.any():
                    val = work_df.loc[mask, tcol].iloc[0]
                else:
                    # fallback: take most recent value <= target_date
                    mask2 = (work_df["countyFIPS"] == c) & (work_df["date"] <= target_date)
                    if mask2.any():
                        val = work_df.loc[mask2, tcol].iloc[-1]
                if pd.isna(val):
                    val = 0.0
                feat_row[lag_name] = val

        # compute derived new_cases_lag features if confirmed_cases lags exist
        feat_row["new_cases_lag1"] = feat_row.get("confirmed_cases_lag1", 0.0) - feat_row.get("confirmed_cases_lag7", 0.0)
        feat_row["new_cases_lag7"] = feat_row.get("confirmed_cases_lag7", 0.0) - feat_row.get("confirmed_cases_lag14", 0.0)

        # For other numeric static features (demographics, tests), bring last known value from work_df if exists else NaN/0
        for col in hist_df.columns:
            if col in ["countyFIPS", "stateFIPS", "date", "confirmed_cases"]:
                continue
            if col in feat_row:  # already set via lag loop
                continue
            # static/demographic features: use historical value for county (most recent)
            mask3 = (work_df["countyFIPS"] == c)
            if mask3.any():
                # pick last non-null
                series = work_df.loc[mask3, col].dropna()
                if not series.empty:
                    feat_row[col] = series.iloc[-1]
                else:
                    feat_row[col] = 0.0
            else:
                feat_row[col] = 0.0

        needed.append(feat_row)

    # Convert to DataFrame
    feat_df = pd.DataFrame(needed)

    # Ensure columns align with feature_cols; if missing column in feat_df, add it with 0
    for c in feature_cols:
        if c not in feat_df.columns:
            feat_df[c] = 0.0

    feat_df = feat_df[feature_cols]  # order columns exactly as during training

    # Make predictions (model expects log1p target)
    pred_log = model.predict(feat_df.values)
    pred_vals = np.expm1(pred_log)
    pred_vals = np.clip(pred_vals, 0, None)
    pred_vals = np.round(pred_vals).astype(int)  # integer counts

    # Attach predictions back to cur_rows in same order
    cur_rows[OUTPUT_TARGET_COL] = pred_vals

    # We'll also ensure per-county sequence in work_df is maintained across dates
    for idx, crow in cur_rows.iterrows():
        c = crow["countyFIPS"]
        pred_val = int(crow[OUTPUT_TARGET_COL])
        # last known cumulative for county in work_df (if any)
        mask = (work_df["countyFIPS"] == c)
        if mask.any():
            last_known = work_df.loc[mask, "confirmed_cases"].iloc[-1]
            if pd.isna(last_known):
                last_known = 0
        cur_rows.at[idx, OUTPUT_TARGET_COL] = pred_val

    # Append predictions to work_df, but we must also keep 'confirmed_cases' column to be used for next-day lags
    # Create a copy in same column layout as hist_df (including confirmed_cases)
    append_rows = cur_rows.copy()
    append_rows[TARGET_COL] = pred_vals   # use pred_vals directly
    append_rows = append_rows[hist_df.columns]

    # Save predictions in the global results list (preserve original pred_template order)
    # We will map them back at the end by matching countyFIPS+date
    for i, row in cur_rows.iterrows():
        predictions.append({
            "countyFIPS": int(row["countyFIPS"]),
            "stateFIPS": int(row["stateFIPS"]) if not pd.isna(row["stateFIPS"]) else np.nan,
            "date": row["date"],
            OUTPUT_TARGET_COL: int(row[OUTPUT_TARGET_COL])
        })

    print(f"Predicted date {cur_date.date()} for {len(cur_rows)} rows.")

Predicted date 2021-02-01 for 3142 rows.
Predicted date 2021-02-02 for 3142 rows.
Predicted date 2021-02-03 for 3142 rows.
Predicted date 2021-02-04 for 3142 rows.
Predicted date 2021-02-05 for 3142 rows.
Predicted date 2021-02-06 for 3142 rows.
Predicted date 2021-02-07 for 3142 rows.
Predicted date 2021-02-08 for 3142 rows.
Predicted date 2021-02-09 for 3142 rows.
Predicted date 2021-02-10 for 3142 rows.
Predicted date 2021-02-11 for 3142 rows.
Predicted date 2021-02-12 for 3142 rows.
Predicted date 2021-02-13 for 3142 rows.
Predicted date 2021-02-14 for 3142 rows.
Predicted date 2021-02-15 for 3142 rows.


In [52]:
pred_df = pd.DataFrame(predictions)
pred_df.head()

,countyFIPS,stateFIPS,date,Confirmed_cases
0,1001,1,2021-02-01,5542
1,1003,1,2021-02-01,18647
2,1005,1,2021-02-01,1986
3,1007,1,2021-02-01,2305
4,1009,1,2021-02-01,5524


In [53]:
pred_template.head()

,Unnamed: 0,countyFIPS,stateFIPS,date,Confirmed_cases
0,1,1001,1,2021-02-01,NaN
1,2,1001,1,2021-02-02,NaN
2,3,1001,1,2021-02-03,NaN
3,4,1001,1,2021-02-04,NaN
4,5,1001,1,2021-02-05,NaN


In [54]:
# -----------------------
# Build final submission by merging predictions back into pred_template preserving order
# -----------------------
pred_df = pd.DataFrame(predictions)

pred_temp = pred_template.drop(columns=['Confirmed_cases'])
# Merge into original pred_template (left join to preserve any extra columns)
final = pred_temp.merge(pred_df, on=["countyFIPS", "stateFIPS", "date"], how="left")

# If any prediction missing (shouldn't), fill with 0
final[OUTPUT_TARGET_COL] = final[OUTPUT_TARGET_COL].fillna(0).astype(int)

final.to_excel(OUTPUT_PATH, index=False)
print(f"\nSaved recursive forecasts to: {OUTPUT_PATH}")


Saved recursive forecasts to: EQ_2021_submission.xlsx
